In [ ]:
# server.py

import logging
import datetime
import time
import psutil
import os
import uuid
from flask import Flask, request, jsonify

app = Flask(__name__)

class Server:
    def __init__(self):
        self.received_data = []  # List of dictionaries with 'device_id', 'timestamp', 'value'
        self.user_data = {}      # {user_id: {'timestamp': ..., 'value': ...}}
        self.setup_logging()

    def setup_logging(self):
        # Configure the logging module
        logging.basicConfig(
            filename='server.log',
            level=logging.INFO,
            format='%(asctime)s %(levelname)s: %(message)s'
        )

    def receive_data(self, device_id, data, request_id):
        # Start CPU and timing measurements
        process = psutil.Process(os.getpid())
        cpu_times_start = process.cpu_times()
        start_time = time.time()

        # Store the received data with timestamp
        entry = {
            'device_id': device_id,
            'timestamp': datetime.datetime.now().isoformat(),
            'value': data
        }
        self.received_data.append(entry)
        logging.info(f"Request ID: {request_id} | Server: Received data from '{device_id}': {data}")

        # End CPU and timing measurements
        end_time = time.time()
        cpu_times_end = process.cpu_times()

        # Calculate CPU time and processing time
        user_cpu_time = cpu_times_end.user - cpu_times_start.user
        system_cpu_time = cpu_times_end.system - cpu_times_start.system
        total_cpu_time = user_cpu_time + system_cpu_time
        processing_time = end_time - start_time

        # Log the performance data
        logging.info(
            f"Request ID: {request_id} | Action: receive_data | Device ID: {device_id} | "
            f"Processing Time: {processing_time:.6f}s | CPU Time: User={user_cpu_time:.6f}s "
            f"System={system_cpu_time:.6f}s Total={total_cpu_time:.6f}s"
        )

        return {'status': 'Data received'}

    def read_data(self, request_id):
        # Start CPU and timing measurements
        process = psutil.Process(os.getpid())
        cpu_times_start = process.cpu_times()
        start_time = time.time()

        # Return all received data
        data = self.received_data
        logging.info(f"Request ID: {request_id} | Server: Data read by a user.")

        # End CPU and timing measurements
        end_time = time.time()
        cpu_times_end = process.cpu_times()

        # Calculate CPU time and processing time
        user_cpu_time = cpu_times_end.user - cpu_times_start.user
        system_cpu_time = cpu_times_end.system - cpu_times_start.system
        total_cpu_time = user_cpu_time + system_cpu_time
        processing_time = end_time - start_time

        # Log the performance data
        logging.info(
            f"Request ID: {request_id} | Action: read_data | Processing Time: {processing_time:.6f}s | "
            f"CPU Time: User={user_cpu_time:.6f}s System={system_cpu_time:.6f}s "
            f"Total={total_cpu_time:.6f}s"
        )

        return data

    def write_data(self, user_id, data, request_id):
        # Start CPU and timing measurements
        process = psutil.Process(os.getpid())
        cpu_times_start = process.cpu_times()
        start_time = time.time()

        # Store data written by the user
        self.user_data[user_id] = {
            'timestamp': datetime.datetime.now().isoformat(),
            'value': data
        }
        logging.info(f"Request ID: {request_id} | Server: User '{user_id}' wrote data: {data}")

        # End CPU and timing measurements
        end_time = time.time()
        cpu_times_end = process.cpu_times()

        # Calculate CPU time and processing time
        user_cpu_time = cpu_times_end.user - cpu_times_start.user
        system_cpu_time = cpu_times_end.system - cpu_times_start.system
        total_cpu_time = user_cpu_time + system_cpu_time
        processing_time = end_time - start_time

        # Log the performance data
        logging.info(
            f"Request ID: {request_id} | Action: write_data | User ID: {user_id} | "
            f"Processing Time: {processing_time:.6f}s | CPU Time: User={user_cpu_time:.6f}s "
            f"System={system_cpu_time:.6f}s Total={total_cpu_time:.6f}s"
        )

        return {'status': 'Data written'}

# Instantiate the Server
server_instance = Server()

# Flask API Endpoints

@app.route('/receive_data', methods=['POST'])
def receive_data_endpoint():
    """
    API endpoint for devices to send data to the server.
    Expects JSON data with 'device_id', 'data', and optional 'request_id'.
    """
    try:
        json_data = request.get_json()
        device_id = json_data.get('device_id')
        data = json_data.get('data')
        request_id = json_data.get('request_id', str(uuid.uuid4()))

        # Input validation
        if not device_id or data is None:
            return jsonify({'error': 'Invalid input data'}), 400

        result = server_instance.receive_data(device_id, data, request_id)

        response = {
            'request_id': request_id,
            'status': result['status']
        }
        return jsonify(response), 200
    except Exception as e:
        logging.error(f"Error in receive_data_endpoint: {e}")
        return jsonify({'error': str(e)}), 500

@app.route('/read_data', methods=['GET'])
def read_data_endpoint():
    """
    API endpoint for users to read data from the server.
    Expects optional 'request_id'.
    """
    try:
        request_id = request.args.get('request_id', str(uuid.uuid4()))
        data = server_instance.read_data(request_id)

        response = {
            'request_id': request_id,
            'data': data
        }
        return jsonify(response), 200
    except Exception as e:
        logging.error(f"Error in read_data_endpoint: {e}")
        return jsonify({'error': str(e)}), 500

@app.route('/write_data', methods=['POST'])
def write_data_endpoint():
    """
    API endpoint for users to write data to the server.
    Expects JSON data with 'user_id', 'data', and optional 'request_id'.
    """
    try:
        json_data = request.get_json()
        user_id = json_data.get('user_id')
        data = json_data.get('data')
        request_id = json_data.get('request_id', str(uuid.uuid4()))

        # Input validation
        if not user_id or data is None:
            return jsonify({'error': 'Invalid input data'}), 400

        result = server_instance.write_data(user_id, data, request_id)

        response = {
            'request_id': request_id,
            'status': result['status']
        }
        return jsonify(response), 200
    except Exception as e:
        logging.error(f"Error in write_data_endpoint: {e}")
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    # Run the Flask app on all interfaces, port 5000
    app.run(host='0.0.0.0', port=5000)